In [1]:
# %%file strategy_1.py

import import_ipynb
from strategy_base import TradeStrategyBase

import pandas as pd
import numpy as np

class TradeStrategy1(TradeStrategyBase):
    """
    Trade Strategy 1: Following the trend.
    When price rises or falls above an absolute threshold x percent (p_change_threshold), default to 2%. 
    Buy and hold for a number of days (s_keep_stock_threshold), default to  20. 
    """
    _p_change_threshold = 0.02
    _keep_stock_threshold = 20
    _profit_array = np.array(0)
    
    def __init__(self, p_change_threshold:float = 0.02, s_keep_stock_threshold:int = 20, should_log:bool = False, should_plot:bool = False):
        """
        Initialize Strategy with given parameters
        :type p_change_threshold: float
        :type s_keep_stock_threshold: int 
        :type should_log: bool
        :type should_plot: bool
        :param p_change_threshold: Absolute threshold for a stock to rise or fall through in a day. E.g. 2% = 0.02 
        :param s_keep_stock_threshold: Number of days to hold a stock. E.g. 20 days = 20
        :param should_log Whether or not log the strategy
        :param should_plot Whether or not plot the strategy
        """
        self._p_change_threshold = p_change_threshold
        self._keep_stock_threshold = s_keep_stock_threshold
        self._should_log = should_log
        self._should_plot = should_plot
        
        self._profit_array = np.array(0)
        self._held_period_days = 0
        
    def trade(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Perform Analysis, Decide on Trade, and Execute Trade
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        :return: 
        """
        
        """
        First, Buying
        """
        if self._evaluate_buying(date=date,
                                 tradeday=tradeday):
            self._execute_buying(date=date,
                                 tradeday=tradeday)
        
        """
        Secondly, Selling
        """
        if self._evaluate_selling(date=date,
                                  tradeday=tradeday):
            self._execute_selling(date=date,
                                  tradeday=tradeday)
            
        """
        Lastly, Increment holding date count
        """
        if self._held_period_days != 0:
            # Is currently holding, attach p_change as profit
            self._held_tradedays.append(tradeday)
            self._profit_array = np.append(self._profit_array, tradeday.p_change)
            # Increment date
            self._held_period_days += 1
        
        
    def _evaluate_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        trade_day: pd.Series = tradeday.copy() # Create a copy() in case we need to change it.
        if self._held_period_days == 0 and \
                np.abs(trade_day.p_change) > self._p_change_threshold:
            # When there is no holding, and rise or fall in price had reached the above pre_defined _p_change_threshold
            return True
        else:
            return False
        
    def _execute_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._should_log:
            print("Bought on {} at ${}.".format(date, tradeday.close))
        self._held_tradedays = [tradeday]
        self._held_period_days += 1 # Start holding date count

    def _evaluate_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._held_period_days >= self._keep_stock_threshold:
            # When holding period exceeds s_keep_stock_threshold, sell stocks
            return True
        else:
            return False
            
    def _execute_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._should_log:
            print("Sold on {} at ${}.".format(date, tradeday.close))
        self._held_period_days = 0  # Reset holding count 
        
    @property
    def trade_profit(self):
        profit_array = self._profit_array/100 + 1
        profit_rate = np.product(profit_array)-1
        return profit_rate
    
    @property
    def p_change_threshold(self):
        return self._p_change_threshold
    
    @property
    def keep_stock_threshold(self):
        return self._keep_stock_threshold

importing Jupyter notebook from strategy_base.ipynb
